In [43]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import re

In [30]:
df= pd.read_excel('./01_raw_data/range/친환경차_조회_2025-08-05.xlsx',sheet_name='전기자동차')

c:\Users\junjo\anaconda3\envs\ev_project2\lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [31]:
df_daily_daily_daily _daily _daily 

,제작사 또는 수입사,모델명,에너지소비효율(km/kWh),1회충전주행거리(km),축전지정격전압/용량(V/Ah),타이어형식(전),구동방식,적용일자,제외일자
0,기아,쏘울 전기자동차(30kWh),5.2,180.0,375 / 80,205/60R16,전륜구동,2019-11-20,NaN
1,기아,쏘울 전기자동차(39.2kWh),5.6,250.0,327 / 120,215/55R17,전륜구동,2019-03-15,NaN
2,기아,쏘울 전기자동차(64kWh),5.4,386.0,356 / 180,215/55R17,전륜구동,2019-03-15,NaN
3,기아,니로 전기자동차,5.3,385.0,356 / 180,215/55R17,전륜구동,2018-07-20,NaN
4,기아,니로 전기자동차,5.5,246.0,327 / 120,215/55R17,전륜구동,2018-07-20,NaN
...,...,...,...,...,...,...,...,...,...
633,닛산,LEAF (23.8kWh),5.0,131.0,360 / 66,215/50R17,전륜구동,2014-12-22,2021-07-01
634,한국GM,스파크 EV,5.6,135.0,360 / 60,185/55R15,전륜구동,2013-09-06,2021-07-01
635,한국GM,스파크 EV,6.0,128.0,360 / 52,185/55R15,전륜구동,2013-09-06,2021-07-01
636,재규어랜드로버,재규어 I-PACE EV400,3.5,333.0,388 / 232,245/50R20,사륜구동,2019-02-20,2021-07-01


In [32]:
# 1) 원본 데이터 로드 (컬럼 '적용일자', '제외일자'를 날짜형으로 파싱)
df_daily_daily_daily_daily_daily_daily_daily_daily_daily_daily_daily_daily = pd.read_excel('./01_raw_data/range/친환경차_조회_2025-08-05.xlsx',sheet_name='전기자동차', parse_dates=['적용일자'])
df[['축전지정격전압(V)', '축전지정격용량(Ah)']] = (
    df['축전지정격전압/용량(V/Ah)']
      .str.split('/', expand=True)
)
df[['축전지정격전압(V)', '축전지정격용량(Ah)']] = df[['축전지정격전압(V)', '축전지정격용량(Ah)']].apply(pd.to_numeric, errors='coerce')

c:\Users\junjo\anaconda3\envs\ev_project2\lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [33]:
df= df.drop(columns=['축전지정격전압/용량(V/Ah)','제외일자', '구동방식'])
df

,제작사 또는 수입사,모델명,에너지소비효율(km/kWh),1회충전주행거리(km),타이어형식(전),적용일자,축전지정격전압(V),축전지정격용량(Ah)
0,기아,쏘울 전기자동차(30kWh),5.2,180.0,205/60R16,2019-11-20,375.0,80.0
1,기아,쏘울 전기자동차(39.2kWh),5.6,250.0,215/55R17,2019-03-15,327.0,120.0
2,기아,쏘울 전기자동차(64kWh),5.4,386.0,215/55R17,2019-03-15,356.0,180.0
3,기아,니로 전기자동차,5.3,385.0,215/55R17,2018-07-20,356.0,180.0
4,기아,니로 전기자동차,5.5,246.0,215/55R17,2018-07-20,327.0,120.0
...,...,...,...,...,...,...,...,...
633,닛산,LEAF (23.8kWh),5.0,131.0,215/50R17,2014-12-22,360.0,66.0
634,한국GM,스파크 EV,5.6,135.0,185/55R15,2013-09-06,360.0,60.0
635,한국GM,스파크 EV,6.0,128.0,185/55R15,2013-09-06,360.0,52.0
636,재규어랜드로버,재규어 I-PACE EV400,3.5,333.0,245/50R20,2019-02-20,388.0,232.0


In [58]:
# --- 타이어 기반 분류 함수 ---
def classify_tire_usage(tire):
    """
    타이어 문자열로부터
      - micro (초소형): 림 지름 ≤ 13인치
      - commercial (상업용): LT 접두사, RxxC, PR
      - passenger (승용): 그 외
    """
    s = str(tire).replace(' ', '').upper()
    # 1) 림 지름 추출
    m = re.search(r'R(\d{1,2})', s)
    if m:
        rim = int(m.group(1))
        if rim <= 13:
            return 'micro'
    # 2) 상업용 조건
    if s.startswith('LT'):
        return 'commercial'
    if re.search(r'R\d+C', s):
        return 'commercial'
    if 'PR' in s:
        return 'commercial'
    # 3) 나머지는 승용
    return 'non_commercial'




def classify_by_tire_type(tire):
    """
    타이어 문자열로부터
      - micro (초소형): 림 지름 ≤ 13인치
      - commercial (상업용): LT 접두사, RxxC, PR
      - passenger (승용): 그 외
    """
    s = str(tire).replace(' ', '').upper()
    # 1) 림 지름 추출
    m = re.search(r'R(\d{1,2})', s)
    if m:
        rim = int(m.group(1))
        if rim <= 13:
            return 'micro'
    # 2) 상업용 조건
    if s.startswith('LT'):
        return 'commercial'
    if re.search(r'R\d+C', s):
        return 'commercial'
    if 'PR' in s:
        return 'commercial'
    # 3) 나머지는 승용
    return 'passenger'

# --- 4) 날짜 순회하며 최근 1년 모델 보관·평균 계산(수정) ---
output_rows = []
start_date = datetime(2018, 1, 1)
end_date   = datetime(2024, 12, 31)
active = []   # Series 목록

current = start_date
while current <= end_date:
    # 오늘 적용된 것 추가
    today = df[df['적용일자'] == current]
    active.extend(today.to_dict('records'))
    
    # 1년 지난 건 제거
    cutoff = current - timedelta(days=365)
    active = [r for r in active if r['적용일자'] > cutoff]
    
    # 상업용 / 비상업용 분할
    comm = [r for r in active if classify_tire_usage(r['타이어형식(전)']) == 'commercial']
    nonc = [r for r in active if classify_tire_usage(r['타이어형식(전)']) == 'non_commercial']
    
    # 평균 계산 (NaN 자동 제외)
    def mean_of(key, records):
        return pd.Series([r[key] for r in records]).mean() if records else None
    
    # 전체 평균
    avg_eff_all   = mean_of('에너지소비효율(km/kWh)', active)
    avg_range_all = mean_of('1회충전주행거리(km)',    active)
    avg_cap_all   = mean_of('축전지정격용량(Ah)',      active)
    
    # 상업용 평균
    avg_eff_comm   = mean_of('에너지소비효율(km/kWh)', comm)
    avg_range_comm = mean_of('1회충전주행거리(km)',    comm)
    avg_cap_comm   = mean_of('축전지정격용량(Ah)',      comm)
    
    # 비상업용 평균
    avg_eff_non   = mean_of('에너지소비효율(km/kWh)', nonc)
    avg_range_non = mean_of('1회충전주행거리(km)',    nonc)
    avg_cap_non   = mean_of('축전지정격용량(Ah)',      nonc)
    
    # 결과 저장
    output_rows.append({
        '날짜': current.strftime('%Y-%m-%d'),
        # 전체
        '평균_에너지소비효율_전체':        avg_eff_all,
        '평균_1회충전주행거리_전체(km)':  avg_range_all,
        '평균_충전지용량_전체(Ah)':        avg_cap_all,
        # 상업용
        '평균_에너지소비효율_상업용':       avg_eff_comm,
        '평균_1회충전주행거리_상업용(km)': avg_range_comm,
        '평균_충전지용량_상업용(Ah)':       avg_cap_comm,
        # 비상업용
        '평균_에너지소비효율_비상업용':     avg_eff_non,
        '평균_1회충전주행거리_비상업용(km)': avg_range_non,
        '평균_충전지용량_비상업용(Ah)':     avg_cap_non,
    })
    
    current += timedelta(days=1)

# --- 5) 결과 저장 ---
df_daily = pd.DataFrame(output_rows)
df_daily.to_csv('./02_interim_data/daily_average.csv', index=False, encoding='utf-8-sig')
df_daily

,날짜,평균_에너지소비효율_전체,평균_1회충전주행거리_전체(km),평균_충전지용량_전체(Ah),평균_에너지소비효율_상업용,평균_1회충전주행거리_상업용(km),평균_충전지용량_상업용(Ah),평균_에너지소비효율_비상업용,평균_1회충전주행거리_비상업용(km),평균_충전지용량_비상업용(Ah)
0,2018-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2552,2024-12-27,4.380292,386.751825,171.960741,2.9,212.846154,163.316667,4.516393,409.52459,173.726446
2553,2024-12-28,4.380292,386.751825,171.960741,2.9,212.846154,163.316667,4.516393,409.52459,173.726446
2554,2024-12-29,4.380292,386.751825,171.960741,2.9,212.846154,163.316667,4.516393,409.52459,173.726446
2555,2024-12-30,4.380292,386.751825,171.960741,2.9,212.846154,163.316667,4.516393,409.52459,173.726446


In [60]:

df_daily['날짜'] = pd.to_datetime(df_daily['날짜'])
df_daily['연월'] = df_daily['날짜'].dt.to_period('M')

monthly = (
    df_daily
    .groupby('연월')
    .mean(numeric_only=True)
    .reset_index()
)

monthly['연월'] = monthly['연월'].dt.to_timestamp()


monthly.to_csv('./02_interim_data/monthly_average.csv', index=False, encoding='utf-8-sig')
monthly

,연월,평균_에너지소비효율_전체,평균_1회충전주행거리_전체(km),평균_충전지용량_전체(Ah),평균_에너지소비효율_상업용,평균_1회충전주행거리_상업용(km),평균_충전지용량_상업용(Ah),평균_에너지소비효율_비상업용,평균_1회충전주행거리_비상업용(km),평균_충전지용량_비상업용(Ah)
0,2018-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-04-01,4.750000,377.000000,220.000000,NaN,NaN,NaN,4.750000,377.000000,220.000000
4,2018-05-01,4.750000,377.000000,220.000000,NaN,NaN,NaN,4.750000,377.000000,220.000000
...,...,...,...,...,...,...,...,...,...,...
79,2024-08-01,4.394767,370.501813,183.989270,2.857143,203.285714,169.169231,4.596659,400.574033,187.493669
80,2024-09-01,4.350446,382.995804,182.884730,2.857143,203.285714,169.169231,4.517480,410.580492,185.733572
81,2024-10-01,4.350410,386.751657,179.831839,2.871429,206.050691,167.490571,4.516022,414.465980,182.434982
82,2024-11-01,4.371028,378.728503,171.667636,2.887556,209.256410,165.541111,4.554270,407.257480,173.509910


In [78]:

# — 1) 연월 → 년/월 분리
df['연월'] = pd.to_datetime(df['연월'])
df['년'] = df['연월'].dt.year
df['월'] = df['연월'].dt.month

# — 2) wide → long (melt)
melted = df.melt(
    id_vars=['연월','년','월'],
    var_name='variable',
    value_name='value'
)

# — 3) ‘평균_’ 접두사 제거 및 차종/측정항목 분리
#    예: '평균_1회충전주행거리_비상업용(km)' → '1회충전주행거리_비상업용(km)'
melted['var2'] = melted['variable'].str.replace(r'^평균_','',regex=True)

#    맨 마지막 '_' 기준으로 split → 측정항목 / 카테고리
melted[['measurement','category_raw']] = melted['var2'].str.rsplit('_', n=1, expand=True)

#    '비상업용(km)' → '비상업용' 과 같이 괄호 뒷부분(unit) 제거
melted['차종'] = melted['category_raw'].str.extract(r'([^\(]+)')

# — 4) measurement를 다시 wide로 pivot
df_long = (
    melted
    .pivot_table(
        index=['년','월','차종'],
        columns='measurement',
        values='value',
        aggfunc='first'
    )
    .reset_index()
)

# — 5) 컬럼 이름 정리 (optional)
df_long = df_long.rename(columns={
    '1회충전주행거리': '주행거리',
    '충전지용량': '충전지용량',
    '에너지소비효율': '에너지소비효율'
})
# 차종명 매핑
mapping = {
    '비상업용': '승용',
    '상업용':   '화물',
    '전체':     '소계'
}

df_long['차종'] = df_long['차종'].map(mapping)
# 결과 확인
df_long.to_csv('./02_interim_data/monthly_long_by_category.csv',
              index=False, encoding='utf-8-sig')
df_long


measurement,년,월,차종,주행거리,에너지소비효율,충전지용량
0,2018,4,승용,377.000000,4.750000,220.000000
1,2018,4,소계,377.000000,4.750000,220.000000
2,2018,5,승용,377.000000,4.750000,220.000000
3,2018,5,소계,377.000000,4.750000,220.000000
4,2018,6,승용,377.000000,4.750000,220.000000
...,...,...,...,...,...,...
226,2024,11,화물,209.256410,2.887556,165.541111
227,2024,11,소계,378.728503,4.371028,171.667636
228,2024,12,승용,408.810841,4.521298,174.553654
229,2024,12,화물,212.846154,2.900000,163.316667


In [70]:
# df_result는 앞서 만든 long-format DataFrame입니다.
# '계'(전체)는 이미 확인하셨으니, 이제 같은 포맷으로 '승용'과 '화물'을 추가로 뽑아봅니다.

import pandas as pd

# (1) 차종별로 나눠서 확인하기
for kind in ['계', '승용', '화물']:
    print(f"\n=== 차종: {kind} ===")
    subset = df_result[df_result['차종'] == kind]
    # 상위 12개 행 미리보기
    print(subset.head(12).to_string(index=False))

# (2) 필요하다면 차종 순서대로 재정렬된 전체 DataFrame 만들기
order = ['계', '승용', '화물']
df_result['차종'] = pd.Categorical(df_result['차종'], categories=order, ordered=True)
df_all = df_result.sort_values(['년', '월', '차종']).reset_index(drop=True)

# (3) 파일로 저장
df_all.to_csv('./02_interim_data/monthly_long_by_category.csv',
              index=False, encoding='utf-8-sig')

# (4) 최종 확인
print("\n=== 재정렬된 전체 데이터 미리보기 ===")
print(df_all.head(36).to_string(index=False))



=== 차종: 계 ===
   년  월 차종  에너지소비효율       주행거리      충전지용량
2018  1  계      NaN        NaN        NaN
2018  2  계      NaN        NaN        NaN
2018  3  계      NaN        NaN        NaN
2018  4  계 4.750000 377.000000 220.000000
2018  5  계 4.750000 377.000000 220.000000
2018  6  계 4.729000 370.760000 218.000000
2018  7  계 4.635115 314.699539 194.470046
2018  8  계 4.785714 314.857143 185.714286
2018  9  계 4.785714 314.857143 185.714286
2018 10  계 4.785714 314.857143 185.714286
2018 11  계 4.740000 325.066667 192.666667
2018 12  계 4.784194 315.570968 183.080645

=== 차종: 승용 ===
Empty DataFrame
Columns: [년, 월, 차종, 에너지소비효율, 주행거리, 충전지용량]
Index: []

=== 차종: 화물 ===
Empty DataFrame
Columns: [년, 월, 차종, 에너지소비효율, 주행거리, 충전지용량]
Index: []

=== 재정렬된 전체 데이터 미리보기 ===
   년  월 차종  에너지소비효율       주행거리      충전지용량
2018  1  계      NaN        NaN        NaN
2018  2  계      NaN        NaN        NaN
2018  3  계      NaN        NaN        NaN
2018  4  계 4.750000 377.000000 220.000000
2018  5  계 4.750000 377.000000 220.

C:\Users\junjo\AppData\Local\Temp\ipykernel_26280\4050745946.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_result['차종'] = pd.Categorical(df_result['차종'], categories=order, ordered=True)
